In [1]:
import numpy as np
from itertools import combinations


### Zaczynamy od Macierzy X, którą standaryzujemy

In [2]:
# Tworzymy losową macierz X (przykład)
macierz_X = np.random.rand(4, 6)  # 3 obserwacje, 4 predyktory

# Obliczamy średnią dla każdej kolumny
srednie_kolumn = np.mean(macierz_X, axis=0)

# Obliczamy wariancję dla każdej kolumny (przy użyciu N, a nie N-1)
wariancje_kolumn = np.mean((macierz_X - srednie_kolumn)**2, axis=0)

# Standaryzujemy kolumny
macierz_X_zestandaryzowana = (macierz_X - srednie_kolumn) / np.sqrt(wariancje_kolumn)

# Weryfikujemy wyniki
srednie_zestandaryzowanych_kolumn = np.mean(macierz_X_zestandaryzowana, axis=0)  # Średnia zestandaryzowanych kolumn
wariancje_zestandaryzowanych_kolumn = np.mean(macierz_X_zestandaryzowana**2, axis=0)  # Wariancja zestandaryzowanych kolumn

print("Średnia zestandaryzowanych kolumn:", srednie_zestandaryzowanych_kolumn)
print("Wariancja zestandaryzowanych kolumn:", wariancje_zestandaryzowanych_kolumn)
print("Normy L2 kolumn:", np.linalg.norm(macierz_X_zestandaryzowana, ord=2, axis=0))  # Norma L2 kolumn
print("Pierwiastek kwadratowy z liczby obserwacji:", np.sqrt(macierz_X.shape[0]))  # Pierwiastek z liczby obserwacji

Średnia zestandaryzowanych kolumn: [ 1.52655666e-16  2.98372438e-16 -3.88578059e-16  5.55111512e-17
 -1.11022302e-16  5.55111512e-17]
Wariancja zestandaryzowanych kolumn: [1. 1. 1. 1. 1. 1.]
Normy L2 kolumn: [2. 2. 2. 2. 2. 2.]
Pierwiastek kwadratowy z liczby obserwacji: 2.0


In [3]:
def parametr_niespojnosci_parami(X):
    liczba_wierszy, liczba_kolumn = X.shape  # X to macierz o wymiarach (n, d): n wierszy i d kolumn
    maksymalna_wartosc = float('-inf')
    for kolumna_j in range(liczba_kolumn):
        for kolumna_k in range(liczba_kolumn):
            # Obliczamy iloczyn skalarny kolumn j i k
            iloczyn_skalarny = abs(np.dot(X[:, kolumna_j], X[:, kolumna_k])) / liczba_wierszy
            if kolumna_j == kolumna_k:
                iloczyn_skalarny -= 1  # Odejmujemy 1, jeśli porównujemy ten sam wektor ze sobą
            
            # Aktualizujemy maksymalną wartość o wartość bezwzględną bieżącego obliczenia
            maksymalna_wartosc = max(maksymalna_wartosc, abs(iloczyn_skalarny))
    return maksymalna_wartosc

In [4]:
delta_pw = parametr_niespojnosci_parami(macierz_X_zestandaryzowana)

In [5]:
delta_pw

np.float64(0.784515227381398)

In [6]:
n, d = 800,900  # n < d
X = np.random.randn(n, d)  # Macierz losowa o niezależnych kolumnach
#standaryzujemy kolumny
X = (X - np.mean(X, axis=0)) / np.std(X, axis=0)

# obliczamy delta_pw dla macierzy X
delta_pw = parametr_niespojnosci_parami(X)
delta_pw

np.float64(0.1749454466137718)

In [7]:
n, d = 8,9  # n < d
X = np.random.randn(n, d)  # Macierz losowa o niezależnych kolumnach
#standaryzujemy kolumny
X = (X - np.mean(X, axis=0)) / np.std(X, axis=0)

# obliczamy delta_pw dla macierzy X
delta_pw = parametr_niespojnosci_parami(X)
delta_pw

np.float64(0.7851762713724241)

In [10]:
def parametr_ograniczonej_izometrii(X, s):
    """
    Sprawdza, czy macierz X spełnia warunek ograniczonej izometrii rzędu s z współczynnikiem delta_s(X).

    Parametry:
    X (np.array): Macierz wejściowa o rozmiarze n x d.
    s (int): Rząd warunku ograniczonej izometrii.
    
    Zwraca:
    float: Współczynnik ograniczonej izometrii delta_s(X), jeśli warunek jest spełniony, w przeciwnym razie -1.
    """
    # Inicjalizujemy współczynnik ograniczonej izometrii jako -1 (oznaczając brak spełnienia warunku)
    delta_s = -1
    
    # Liczba kolumn w macierzy X to wymiar d
    liczba_wierszy, liczba_kolumn = X.shape
    
    # Sprawdzamy, czy s znajduje się w prawidłowym zakresie
    if s < 1 or s > liczba_kolumn:
        raise ValueError("Rząd s musi mieścić się w zakresie od 1 do liczby kolumn d w macierzy X.")
    
    # Generujemy wszystkie podzbiory S o rozmiarze co najwyżej s
    wszystkie_podzbiory = [np.array(comb) for k in range(1, s+1) for comb in combinations(range(liczba_kolumn), k)]
    
    # Obliczamy współczynnik ograniczonej izometrii dla wszystkich podzbiorów S o rozmiarze co najwyżej s
    for podzbior in wszystkie_podzbiory:
        # Wyodrębniamy podmacierz X_S odpowiadającą kolumnom z podzbioru S
        X_S = X[:, podzbior]
        # Obliczamy iloczyn macierzy X_S^T X_S i normalizujemy przez n
        iloczyn = X_S.T @ X_S / liczba_wierszy
        # Obliczamy maksymalna_wartosc_wlasna
        maksymalna_wartosc_wlasna = np.linalg.norm(iloczyn - np.eye(len(podzbior)), ord=2)
        
        # Aktualizujemy współczynnik ograniczonej izometrii, jeśli wartość jest większa
        delta_s = max(delta_s, maksymalna_wartosc_wlasna)
    
    return delta_s

-   Jeśli $\delta_s(X)$ jest małe (np.  $\ll 1$), to każda podmacierz o wymiarze s zachowuje się prawie jak macierz ortogonalna.
-   Jeśli $\delta_s(X)$ jest duże (np. $>1$), to istnieją podzbiory kolumn macierzy X, które są silnie skorelowane, co powoduje problemy numeryczne.

In [11]:
delta_s_2 = parametr_ograniczonej_izometrii(macierz_X_zestandaryzowana, s=2)

In [12]:
delta_s_2

np.float64(0.7845152273813982)

In [13]:
delta_pw = parametr_niespojnosci_parami(macierz_X_zestandaryzowana)
delta_pw

np.float64(0.784515227381398)

### jak widać dla s=2 wielkości $\delta_s(X)$ oraz $\delta_pw(X)$ są takie same.

In [17]:
delta_s_1 = parametr_ograniczonej_izometrii(macierz_X_zestandaryzowana, s=1)

In [18]:
delta_s_1

np.float64(2.220446049250313e-16)

### Dla $s=1$ mamy $\delta_s(X)=0$ co pokazuje, ze norma druga kolumn nalezy do przedziału $(1-\delta_1(X), 1+\delta_1(X))$.

In [19]:
print(macierz_X_zestandaryzowana)

[[ 1.31371992e+00  9.17417301e-01 -9.74177601e-01 -1.56053852e+00
   1.30767746e-01 -5.54852247e-02]
 [-1.44675021e+00 -1.53318123e+00 -5.68551738e-04 -1.05736437e-01
  -2.70036048e-01 -1.57969065e+00]
 [-2.26917372e-01  8.63812803e-01  1.62225471e+00  1.12062940e+00
   1.46610873e+00  5.30709237e-01]
 [ 3.59947659e-01 -2.48048870e-01 -6.47508558e-01  5.45645560e-01
  -1.32684043e+00  1.10446664e+00]]


In [20]:
delta_s_3 = parametr_ograniczonej_izometrii(macierz_X_zestandaryzowana, s=3)

In [21]:
delta_s_3

np.float64(1.366594279563812)

In [22]:
def znajdz_najgorszy_podzior(X, s):
    """
    Znajduje podzbiór kolumn macierzy X o wymiarze co najwyżej s, dla którego 
    współczynnik ograniczonej izometrii (delta_s) jest największy.

    Parametry:
    X (np.array): Macierz wejściowa o rozmiarze (n, d).
    s (int): Maksymalna liczba kolumn w podzbiorze.

    Zwraca:
    tuple: (Najbardziej problematyczne kolumny, Największa wartość delta_s)
    """
    n, d = X.shape
    
    if s < 1 or s > d:
        raise ValueError("s musi mieścić się w zakresie od 1 do liczby kolumn d w macierzy X.")

    # Generowanie wszystkich podzbiorów S o rozmiarze dokładnie s
    wszystkie_podzbiory = [np.array(comb) for comb in combinations(range(d), s)]
    
    najgorszy_podzior = None
    najgorsza_wartosc = -1

    # Iterujemy przez wszystkie podzbiory i znajdujemy największą wartość
    for podzbior in wszystkie_podzbiory:
        X_S = X[:, podzbior]
        iloczyn = X_S.T @ X_S / n  # Normalizowany iloczyn wewnętrzny kolumn
        maksymalna_wartosc_wlasna = np.linalg.norm(iloczyn - np.eye(len(podzbior)), ord=2)
        
        if maksymalna_wartosc_wlasna > najgorsza_wartosc:
            najgorsza_wartosc = maksymalna_wartosc_wlasna
            najgorszy_podzior = podzbior

    return najgorszy_podzior, najgorsza_wartosc

In [23]:
najgorszy_podzior, najgorsza_wartosc = znajdz_najgorszy_podzior(macierz_X_zestandaryzowana, s=3)
print("Najbardziej problematyczne kolumny:", najgorszy_podzior)
print("Największa wartość dla tego podzbioru:", najgorsza_wartosc)

Najbardziej problematyczne kolumny: [0 1 5]
Największa wartość dla tego podzbioru: 1.366594279563812


In [24]:
delta_s_4 = parametr_ograniczonej_izometrii(macierz_X_zestandaryzowana, s=4)

In [25]:
delta_s_4

np.float64(1.437379043046473)

In [26]:
najgorszy_podzior, najgorsza_wartosc = znajdz_najgorszy_podzior(macierz_X_zestandaryzowana, s=4)
print("Najbardziej problematyczne kolumny:", najgorszy_podzior)
print("Największa wartość dla tego podzbioru:", najgorsza_wartosc)

Najbardziej problematyczne kolumny: [0 2 3 4]
Największa wartość dla tego podzbioru: 1.437379043046473


In [27]:
X = np.array([
    [1, 0, 1, 2, 1, 3, 0, 2, 3, 4, 0, 2, 1, 3, 4, 2, 0, 1, 4, 3],
    [0, 1, 2, 1, 3, 0, 2, 3, 4, 1, 2, 0, 3, 1, 2, 4, 3, 0, 1, 2],
    [2, 3, 1, 0, 4, 1, 3, 2, 0, 1, 4, 3, 2, 1, 0, 4, 2, 3, 1, 0],
    [3, 4, 0, 2, 1, 2, 3, 4, 1, 0, 3, 2, 4, 1, 3, 0, 2, 1, 4, 3],
    [4, 1, 3, 2, 0, 1, 4, 3, 2, 0, 1, 2, 3, 4, 0, 1, 3, 2, 4, 0],
    [1, 2, 3, 0, 4, 2, 1, 3, 0, 4, 3, 1, 2, 4, 0, 1, 3, 4, 2, 0],
    [3, 0, 1, 2, 4, 1, 2, 3, 0, 4, 1, 2, 0, 3, 4, 2, 1, 3, 4, 0],
    [2, 4, 0, 3, 1, 2, 1, 4, 3, 0, 2, 1, 3, 4, 0, 1, 3, 2, 4, 0],
    [4, 3, 2, 1, 0, 4, 3, 2, 1, 0, 4, 3, 2, 0, 1, 3, 4, 0, 2, 1],
    [1, 3, 4, 0, 2, 1, 2, 3, 4, 0, 1, 3, 2, 4, 0, 1, 3, 4, 2, 0],
    [2, 0, 3, 1, 4, 2, 1, 3, 4, 0, 2, 1, 4, 3, 0, 2, 1, 3, 4, 0],
    [3, 2, 4, 1, 0, 3, 2, 1, 4, 0, 3, 1, 4, 2, 0, 3, 1, 4, 2, 0],
    [0, 4, 2, 1, 3, 0, 4, 2, 1, 3, 0, 4, 1, 3, 2, 0, 4, 3, 2, 1],
    [1, 3, 0, 4, 2, 1, 3, 0, 4, 2, 1, 3, 4, 0, 2, 1, 3, 4, 2, 0],
    [2, 0, 4, 1, 3, 2, 0, 4, 1, 3, 2, 0, 3, 4, 1, 2, 0, 4, 1, 3]
])

# Normalizacja kolumn (standaryzacja)
X = (X - np.mean(X, axis=0)) / np.std(X, axis=0)

In [28]:
delta_s_3 = parametr_ograniczonej_izometrii(X, s=3)
delta_s_3

np.float64(1.2686977996038404)

In [29]:
n, d = 8,9  # n < d
X = np.random.randn(n, d)  # Macierz losowa o niezależnych kolumnach
#standaryzujemy kolumny
X = (X - np.mean(X, axis=0)) / np.std(X, axis=0)

In [30]:
delta_s_3 = parametr_ograniczonej_izometrii(X, s=3)
delta_s_3

np.float64(1.106779919611106)